In [ ]:
chrom = None

In [1]:
import pyspark
import dxpy
import hail as hl

WD='/opt/notebooks'

In [2]:
my_database = dxpy.find_one_data_object(
    name="my_database", 
    project=dxpy.find_one_project()["id"]
)["id"]
database_dir = f'dnax://{my_database}'
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)
hl.init(sc=sc, tmp_dir=f'{database_dir}/tmp/')

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.4
SparkUI available at http://ip-10-60-94-127.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.78-b17627756568
LOGGING: writing to /opt/notebooks/hail-20220818-1344-0.2.78-b17627756568.log


## S0. Define functions, load data

In [28]:
def get_final_filter_mt_path(chrom):
    return f'{database_dir}/04_final_filter_write_to_mt/ukb_wes_450k.qced.chr{chrom}.mt'

def export_table(ht, fname, out_folder):
    ht.naive_coalesce(1).export(f'file:///opt/notebooks/{fname}')

    dxpy.upload_local_file(
        filename=f'/opt/notebooks/{fname}',
        name=fname,
        folder=out_folder,
        parents=True
    )

## S2. Hail `variant_qc`

In [ ]:
%%time

mt = hl.read_matrix_table(get_final_filter_mt_path(chrom=chrom))

mt = hl.variant_qc(mt)

rows = mt.rows()
rows = rows.transmute(**rows.variant_qc.flatten())

export_table(
    ht=rows,
    fname=f'variant_qc.final_stats.c{chrom}.tsv.gz',
    out_folder='/data/05_final_stats'
)